## Prerequisite
Install boto3 library. You will need it to test IRSA. Some of the assumptions made in this enablement scenario-

1. EKS Account Id - `946429944765` Prod-Field deployment is in this AWS Account
2. AWS Assets Account Id - `524112250363` This is the account where your corporate assets like S3 buckets containing your corporate data are placed. 
   It would be ideal if you have some access to this account. But we can still run through most of this tutorial without it.
3. The OIDC Provider Attached to your EKS cluster is ` oidc.eks.us-west-2.amazonaws.com/id/6377798F9D92DFC7C9C3873B8DA27C68`
   If the prod-field deployments EKS cluster changes, so will this provider. And the IRSA service will need to be reinstalled.


This project is based on the IRSA enablement [repository](https://github.com/cerebrotech/field-engineered-projects-enablement/tree/main/IRSA)

https://github.com/cerebrotech/field-engineered-projects-enablement/tree/main/IRSA

### Verify the services are running


In [ ]:
#First let us verify that the IRSA backend service is running
!curl http://irsa-svc.domino-field/healthz

In [ ]:
!curl http://localhost:6003/healthz

## Web Identity Token
A Web Identity Token is how your workspace authenticates to AWS IAM. It is injected into your workspace via a `domsed` mutation.

```yaml
  - name: aws-user-token
    projected:
      defaultMode: 422
      sources:
      - serviceAccountToken:
          path: token
          expirationSeconds: 86400
          audience: sts.amazonaws.com
```

Note that this token expires every 24 hours. We can and should make this shorter in a production deploymen. K8s rotates
this token 5 mins prior to expiry. If this token is compromised the attacker can use it to assume roles in the AWS
account using just an `.aws/config` file which matches the one in the workspace from which this token was stolen.

The mutation also injects to AWS specific environment variables
1. AWS_WEB_IDENTITY_TOKEN_FILE
2. AWS_CONFIG_FILE

In [ ]:
!cat $AWS_WEB_IDENTITY_TOKEN_FILE

Now go to www.jwt.io and paste the above token. Note the `iss`. This is the token that boto3 api uses to implicitly authenticate with IAM. This occurs implicitly when the `AWS_WEB_IDENTITY_TOKEN_FILE` environment variable set.

Note the `iss`. This is the token that boto3 api uses to implicitly authenticate with IAM. This occurs implicitly when the `AWS_WEB_IDENTITY_TOKEN_FILE` environment variable set.

Note the `sub` is `system:serviceaccount:domino-compute:run-$DOMINO_RUN_ID` (your workspace service account is same as your run-id)
[Ex. `system:serviceaccount:domino-compute:run-646d27b817c28b5ea4b5b0c1`] 

`system:serviceaccount:domino-compute:run-646d27b817c28b5ea4b5b0c1` (your workspace service account) . This will be important later.


```
{
  "aud": [
    "sts.amazonaws.com"
  ],
  "exp": 1685030736,
  "iat": 1684944336,
  "iss": "https://oidc.eks.us-west-2.amazonaws.com/id/6377798F9D92DFC7C9C3873B8DA27C68",
  "kubernetes.io": {
    "namespace": "domino-compute",
    "pod": {
      "name": "run-646d27b817c28b5ea4b5b0c1-fxc2q",
      "uid": "63e373b0-4d62-4860-a329-baec1afcd3f8"
    },
    "serviceaccount": {
      "name": "run-646d27b817c28b5ea4b5b0c1",
      "uid": "ed97cfcf-58c6-4c1e-8929-71ff9bc335aa"
    }
  },
  "nbf": 1684944336,
  "sub": "system:serviceaccount:domino-compute:run-646d27b817c28b5ea4b5b0c1"
}
```

### AWS Config File

The `boto3` library looks for the config file in the following location `$AWS_CONFIG_FILE` which in your workspace
is set to `/var/run/.aws/config`

In [ ]:
!cat $AWS_CONFIG_FILE

This is file is created by the side-car container during startup. You can always refresh it using the command 
```shell
curl http://localhost:6003/refresh
```


For each profile in the asset account there is a corresponding role in the eks account

| AWS Profile (Asset Account)     | SRC AWS Profile (EKS Account) |
| ----------- | ----------- |
| `acme-list-bucket-role`      | `src_acme-list-bucket-role`       |
| `acme-read-bucket-role`  | `src_acme-read-bucket-role`       |
| `acme-update-bucket-role`   | `src_acme-read-bucket-role`       |

An example `src` profile looks like this:

```shell
[profile src_acme-list-bucket-role]
web_identity_token_file = /var/run/secrets/eks.amazonaws.com/serviceaccount/token
role_arn=arn:aws:iam::946429944765:role/acme-list-bucket-role
```
The corresponding asset profile looks like this:

```shell
[profile acme-list-bucket-role]
source_profile = src_acme-list-bucket-role
role_arn=arn:aws:iam::524112250363:role/acme-list-bucket-role
```

When the user assumes the asset profile `acme-list-bucket-role` it knows its `source_profile` is `src_acme-list-bucket-role`

The `src_acme-list-bucket-role` uses the `web_identity_token_file` to authenticate with AWS IAM which already has 
a trust relationship established with the OIDC provider who issues the web identity token. It uses this to assume the role
`role_arn` in the source profile `aws:iam::946429944765:role/acme-list-bucket-role`

The `aws:iam::946429944765:role/acme-list-bucket-role` in turn tries to assume the role `arn:aws:iam::524112250363:role/acme-list-bucket-role`
which is listed as the `role_arn` in the `[profile acme-list-bucket-role]`

### Bringing it all together

When the workspace starts up, the side-car invokes the backend IRSA service using the following code-snippet

In [ ]:
#Emulate Side-Car
import requests
import os
access_token_endpoint='http://localhost:8899/access-token'
resp = requests.get(access_token_endpoint)


token = resp.text
headers = {
             "Content-Type": "application/json",
             "Authorization": "Bearer " + token,
        }
endpoint='http://irsa-svc.domino-field/map_iam_roles_to_pod_sa'
data = {"run_id": os.environ['DOMINO_RUN_ID']} ## It fetches this fom the downward api
resp = requests.post(endpoint,headers=headers,json=data)
resp.text

The backend IRSA service updates the trust policy file attached to each of the source roles being added to the config file.
The trust policy will look like this:
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::946429944765:oidc-provider/oidc.eks.us-west-2.amazonaws.com/id/6377798F9D92DFC7C9C3873B8DA27C68"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringLike": {
                    "oidc.eks.us-west-2.amazonaws.com/id/6377798F9D92DFC7C9C3873B8DA27C68:aud": "sts.amazonaws.com",
                    "oidc.eks.us-west-2.amazonaws.com/id/6377798F9D92DFC7C9C3873B8DA27C68:sub": [
                        "*:domino-compute:run-646d27b817c28b5ea4b5b0c1",
                        "*:domino-compute:run-646f604955daf764547c5b77"
                    ]
                }
            }
        }
    ]
}
```

Note the `aud` and `sub` fields. This is the reason why the `src_acme-list-bucket-role` profile can assume the role
`arn:aws:iam::946429944765:role/acme-list-bucket-role`. If you remove the entries for your workspace from the trust policy,
the web identity token can authenticate with the AWS IAM but IAM will not allow it to assume the role.

Last each of the roles in the EKS Account have the following policies attached to them:

| Role    | Policies  |
| ----------- | ----------- |
| `acme-list-bucket-role`      | `acme-list-bucket-policy`       |
| `acme-read-bucket-role`  | `acme-read-bucket-policy`       |
| `acme-update-bucket-role`   | `acme-read-bucket-policy`       |

`acme-list-bucket-policy` looks like this:
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sts:AssumeRole",
            "Resource": "arn:aws:iam::524112250363:role/acme-list-bucket-role"
        }
    ]
}
```
The other two policies are similar


To summarize two things make it possible for the workspace to assume the role `arn:aws:iam::946429944765:role/acme-list-bucket-role`
(and the other two roles):

1. The web identity token which was issued by the oidc provider (`sub` provides the identity)
2. The `Condition` section of the trust policy attached to the roles which explictly declares which `sub` are allowed to 
assume the role.
3. The permission policies attached to the roles in the EKS account will be allowed to assume roles in the Assets account.

One last thing, the corresponding roles in the Assets Account ex. `arn:aws:iam::524112250363:role/acme-list-bucket-role`
need to have their trust policy to allow principals in the EKS account to assume it. This is the trust policy attached to
each of the roles `acme-list-bucket-role` , `acme-read-bucket-role` and `acme-update-bucket-role` 

```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::946429944765:root"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```

#### Install boto3 library


In [ ]:
! pip install boto3

### Now lets get some action

Let us actual make calls using the `boto3` api now :


### Listing Profiles

Let us use the boto3 library to fetch all the profiles available. This is similar to `cat $AWS_CONFIG_FILE`

In [ ]:
import boto3.session
for profile in boto3.session.Session().available_profiles:
    print(profile)

### Listing the bucket

First lets verify what the identity of the profile is. This is determines by a combination of the web identity token
and the underlying assumed role in the Assests Account via the role in the EKS Account

In [ ]:
import boto3.session
list_bucket_profile_name='acme-list-bucket-role'
session = boto3.session.Session(profile_name=list_bucket_profile_name)
sts_client = session.client('sts')
sts_client.get_caller_identity()

The following permission policy is attached to the role is `acme-list-bucket-policy` (only relevant snippet)

```json
        {
            "Effect": "Allow",
            "Action": ["s3:ListBucket"],
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket"]
        }
```

In [ ]:
#Now list the bucket
import boto3
test_bucket='domino-acme-test-bucket'
list_bucket_profile_name='acme-list-bucket-role'
session = boto3.session.Session(profile_name=list_bucket_profile_name)
s3_client = session.client('s3')
for key in s3_client.list_objects(Bucket=test_bucket)['Contents']:
    print(key)

### Writing to the bucket

Now we write to the bucket. For that we need to use the profile 'acme-update-bucket-role'. This role in the assets
account will have the following policies attached to it (List, Read, Update permission)

1. `acme-list-bucket-policy` [Only the relevant snipper below]


2. `acme-read-bucket-policy` 

```json
        {
            "Effect": "Allow",
            "Action": ["s3:Get*"],
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        }
```

2. `acme-update-bucket-policy` 

```json
     [
        {
            "Sid": "WriteObjectsInBucket",
            "Effect": "Allow",
            "Action": "s3:PutObject",
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        },
        {
            "Sid": "DeleteObjectsInBucket",
            "Effect": "Allow",
            "Action": "s3:DeleteObject",
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        }
    ]
```

Now let us create a object with key `$DOMINO_STARTING_USERNAME.txt`

In [ ]:
import boto3
import os
test_bucket='domino-acme-test-bucket'
starting_user = os.environ['DOMINO_STARTING_USERNAME']
update_bucket_profile_name='acme-update-bucket-role'
session = boto3.session.Session(profile_name=update_bucket_profile_name)
s3_client = session.client('s3')
object_data = "This is a random string."
object_key = f'{starting_user}.txt'
s3_client.put_object(Body=object_data, Bucket=test_bucket, Key=object_key)

If you try to read the key using this role using the code below you will see an error indicating that the role does not have `GetObject` operation allowed for it.
```shell
ClientError: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied
```


### Reading a key from the bucket

Let us now read the newly added object from the bucket. This is provided by the role `acme-read-bucket-role`. This role
has the  policies `acme-read-bucket-policy` attached to it:

1. `acme-list-bucket-policy`
2. `acme-read-bucket-policy`
```json
         {
            "Sid": "WriteObjectsInBucket",
            "Effect": "Allow",
            "Action": ""s3:Get*"",
            "Resource": ["arn:aws:s3:::domino-acme-test-bucket/*"]
        }
```

Run the following code snippet:

In [31]:
import boto3
import os
test_bucket='domino-acme-test-bucket'
starting_user = os.environ['DOMINO_STARTING_USERNAME']
read_bucket_profile_name='acme-read-bucket-role'
object_key = f'{starting_user}.txt'
session = boto3.session.Session(profile_name=read_bucket_profile_name)
s3_client = session.client('s3')
data = s3_client.get_object(Bucket=test_bucket, Key=object_key)
contents = data['Body'].read()
print(f'\n---Contents of the key {object_key}----\n')
print(contents.decode("utf-8"))


---Contents of the key sameer_wadkar.txt----

This is a random string.


### Deleting a key from the bucket

Finally let us delete the newly created object

In [32]:
import boto3
import os
test_bucket='domino-acme-test-bucket'
update_bucket_profile_name='acme-update-bucket-role'
starting_user = os.environ['DOMINO_STARTING_USERNAME']
object_key = f'{starting_user}.txt'
print(f'Deleting Key {object_key} from bucket {test_bucket}')

session = boto3.session.Session(profile_name=update_bucket_profile_name)
s3_client = session.client('s3')
s3_client.delete_object(Bucket=test_bucket, Key=object_key)
print('\nNow listing bucket:\n')
for key in s3_client.list_objects(Bucket=test_bucket)['Contents']:
    print(key)


Deleting Key sameer_wadkar.txt from bucket domino-acme-test-bucket

Now listing bucket:

{'Key': 'a.txt', 'LastModified': datetime.datetime(2023, 5, 23, 21, 8, 28, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'ops+aws-domino-cs-eval', 'ID': 'c745a304f2b9d04fceed84743d0486eda1ca6aaef0a5a7b560ffbd66a16d8441'}}
{'Key': 'b.txt', 'LastModified': datetime.datetime(2023, 5, 23, 21, 8, 28, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'ops+aws-domino-cs-eval', 'ID': 'c745a304f2b9d04fceed84743d0486eda1ca6aaef0a5a7b560ffbd66a16d8441'}}
{'Key': 'c.txt', 'LastModified': datetime.datetime(2023, 5, 23, 21, 8, 27, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'Size': 0, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'ops+aws-domino-cs-eval', 'ID': 'c745a304f2b9d04fceed84743d0486eda1ca6aaef0a5a7b560ffbd66a16d8441